# 🤗 x 🦾: Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [2]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 39766, done.
remote: Counting objects: 100% (774/774), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 39766 (delta 598), reused 362 (delta 359), pack-reused 38992 (from 2)
Receiving objects: 100% (39766/39766), 170.07 MiB | 36.06 MiB/s, done.
Resolving deltas: 100% (25824/25824), done.
Filtering content: 100% (45/45), 69.03 MiB | 62.83 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge
    ca-

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: soumo-roy2021 (eshaan-rithesh2023-vit-chennai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Install SmolVLA dependencies

In [1]:
!cd lerobot && pip install -e ".[smolvla]"

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.4 MB/s eta 0:00:00
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-0.editable-py3-none-any.whl size=15290 sha256=a31dfd465156f5bb33cd92235fcf9966048a1db968d2f449dc567f45acd8255b
  Stored in directory: /tmp/pip-ephem-wheel-cache-wfqoy6a9/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=047dce74802a7cae160e177931db2d9c7ec3b477a64af785ade79c7113bf541d
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b7

## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [5]:
!rm -rf src/lerobot/outputs/train/my_smolvla
print("removed old diretory")
!cd lerobot && python src/lerobot/scripts/lerobot_train.py \
  --policy.path=cijerezg/smolvla-test \
  --policy.repo_id=s0um0r0y/my-smolvla-model \
  --dataset.repo_id=s0um0r0y/record-test \
  --batch_size=64 \
  --steps=20000 \
  --output_dir=outputs/train/my_smolvla \
  --job_name=my_smolvla_training \
  --policy.device=cuda \
  --wandb.enable=true \
  # --resume=True
  # --config_path=outputs/train/checkpoints/last/pretrained_model/model.safetensors


removed old diretory
WARNING 2025-09-28 17:51:37 ils/utils.py:47 No accelerated backend detected. Using default cpu, this will be slow.
WARNING 2025-09-28 17:51:37 /policies.py:79 Device 'cuda' is not available. Switching to 'cpu'.
WARNING 2025-09-28 17:51:37 ils/utils.py:47 No accelerated backend detected. Using default cpu, this will be slow.
WARNING 2025-09-28 17:51:37 /policies.py:79 Device 'cuda' is not available. Switching to 'cpu'.
INFO 2025-09-28 17:51:37 ot_train.py:148 {'batch_size': 64,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                         

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [6]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `LiquidOS` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/

In [7]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/huggingface_cli.py", line 61, in main
    service.run()
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/upload.py", line 211, in run
    print(self._upload())
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/upload.py", line 271, in _upload
    raise FileNotFoundError(f"No such file or directory: '{self.local_path}'.")
FileNotFoundError: No such file or directory: '/content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model'.
